<a href="https://colab.research.google.com/github/celinaLind/money-honey/blob/main/Financial_Analysis_%26_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Financial Analysis & Automation with LLMs Project

# Install Libraries

In [ ]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

In [ ]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [ ]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'Sector': stock_info.get('sector', 'Information not available')
    }

    return properties

In [ ]:
data = yf.Ticker("NVDA")
stock_info = data.info
print(stock_info)

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment co

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

Cosine similarity between the two sentences: 0.6133


In [ ]:
aapl_info = get_stock_info('AAPL')
print(aapl_info)

{'Ticker': 'AAPL', 'Name': 'Apple Inc.', 'Business Summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experien

In [ ]:
aapl_description = aapl_info['Business Summary']

company_description = "I want to find companies that make smartphones and are headquarted in California"

similarity = cosine_similarity_between_sentences(aapl_description, company_description)

Cosine similarity between the two sentences: 0.3635


# Get all the Stocks in the Stock Market

First, we need to get the symbols (also known as tickers) of all the stocks in the stock market


In [ ]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [ ]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [ ]:
len(company_tickers)

9998

# Inserting Stocks into Pinecone

**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "stocks" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-12-02 at 4 48 03 PM](https://github.com/user-attachments/assets/13b484da-cd00-4337-a4db-4779080eb42c)


**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**


![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)




In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-30-8d5c0f356cf6>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


# Sequential Processing

It will take very long to process all the stocks like this!

*Processing items ONE AT A TIME, the next process item has to wait the previous process to be complete before it can start.

[![](https://mermaid.ink/img/pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4?type=png)](https://mermaid.live/edit#pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4)

In [ ]:
for idx, stock in company_tickers.items():
    stock_ticker = stock['ticker']
    stock_data = get_stock_info(stock_ticker)
    stock_description = stock_data['Business Summary']

    print(f"Processing stock {idx} / {len(company_tickers)} :", stock_ticker)

    vectorstore_from_documents = PineconeVectorStore.from_documents(
        documents=[Document(page_content=stock_description, metadata=stock_data)],
        embedding=hf_embeddings,
        index_name=index_name,
        namespace=namespace
    )

# Parallelizing

[![](https://mermaid.ink/img/pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8?type=png)](https://mermaid.live/edit#pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8)

Can process items in batches. Above it is showing the process of 5 items that once they are complete and confirmed it will move on to the next batch of items.

### QA:
1. EX. Batch1 = A,B,C  Batch2 = D,E,F <br>
Q: **If A finishes first does it have to wait until the rest of batch one is complete before it moves onto an item from batch2?** <br>
A: Each batch has 3 workers, for batch one the workers are simultaneously working on tasks A,B,C. Once one worker completes its task the worker then starts on a task in the
 next batch right away. The worker working on task A will start working on task D once task A is complete and stored.

2. Q: **What is the correlation between the number of workers and cores?** <br>
A: In batch processing, the correlation between the number of workers and the number of CPU cores is generally a direct relationship; meaning, ideally, you should set the number of workers to be roughly equal to the number of available cores on your system to maximize parallel processing efficiency and achieve the best performance for batch jobs. (This also depends if there is multi-threading involved)

In [ ]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

Loaded 3829 successful tickers
No existing unsuccessful tickers file found


In [ ]:
import time
processed_idx = 0

In [ ]:
def process_stock(stock_ticker: str) -> str:

    # Skip if already processed
    if stock_ticker in successful_tickers:
      return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        stock_description = "No description provided" if stock_data['Business Summary'] is None else stock_data['Business Summary']

        print(f"Processing stock: {stock_ticker}",end="\r")
        # Store stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
          )
        print(f"Processed stock: {stock_ticker}")

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, idx: int, max_workers: int = 10) -> None:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(f"{result} => {idx} / {len(tickers)-1}")

                if idx % 500 == 0:
                  print("Sleeping for 2 minutes to avoid rate limiting...")
                  time.sleep(120)
                idx += 1

                # Stop on error
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, processed_idx, max_workers=6)

Already processed MA => 0 / 9997
Sleeping for 2 minutes to avoid rate limiting...
Processed stock: ELMD
Processed stock: TG
Processed stock: FGPR
Processed stock: PSF
Processed stock: AUDC
Processed stock: RBTK
Processed stock: ZNTL
Processed stock: RLEA
Processed stock: ELDN
Processed stock: EVBN
Processed stock: PGEN
Processed stock: EVI
Processed stock: LFMD
Processed stock: TCI
Processed stock: TLS
Processed stock: NPV
Processed stock: CYBN
Processed stock: MBCN
Processed stock: LFCR
Processed stock: ILPT
Processed stock: TSI
Processed stock: BMEA
Processed stock: INGN
Processed stock: ONIT
Processed stock: HIE
Processed stock: MVST
Processed stock: LCNB
Processed stock: TBRG
Processed stock: EVM
Processed stock: STRO
Processed stock: NCZ
Processed stock: CFFI
Processed stock: FUND
Processed stock: FINW
Processed stock: LXEO
Processed stock: MODV
Processed stock: TCRX
Processed stock: CRNT
Processed stock: EVC
Processed stock: MFM
Processed stock: MUE
Processed stock: MVT
Processed

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FACT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FACT&crumb=bhSjaOmA8q8


Processed stock: GRX
Processed stock: ESHA
Processed stock: FLD
Processed stock: NNY
Processed stock: DRTS
Processed stock: MDV
Processed stock: LDDD
Processed stock: CKPT
Processed stock: ITRG
Processed stock: PIM
Processed stock: BMN
Processed stock: ACU
Processed stock: NIU
Processed stock: MMLP
Processed stock: AMLI
Processed stock: WILC
Processed stock: DLNG
Processed stock: CBNA
Processed stock: BFIN
Processed stock: MHF
Processed stock: NUTX
Processed stock: RCKY
Processed stock: STHO
Processed stock: HCVI
Processed stock: EHI
Processed stock: NMG
Processed stock: TNYA
Processed stock: TUSK
Processed stock: BROG
Processed stock: MPTI
Processed stock: BFAC
Already processed SKYW => 1001 / 9997
Already processed BHVN => 1002 / 9997
Already processed ESE => 1003 / 9997
Already processed TEO => 1004 / 9997
Already processed YOU => 1005 / 9997
Already processed ITGR => 1006 / 9997
Already processed EBC => 1007 / 9997
Already processed HAE => 1008 / 9997
Already processed SAM => 1009 

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PMVC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PMVC&crumb=bhSjaOmA8q8


Processed stock: SAML
Processed stock: REOS
Processed stock: IMII
Processed stock: XPON
Processed stock: FECOF
Processed stock: SVRE
Processed stock: WAST
Processed stock: SGBX
Processed stock: CWBR
Processed stock: NGTF
Processed stock: WORX
Processed stock: FSTJ
Processed stock: IPSI
Processed stock: TC
Processed stock: HWKE
Processed stock: BLMH
Processed stock: OMQS
Processed stock: SRSG
Processed stock: ILST
Processed stock: HHHEF
Processed stock: LGCP
Processed stock: LGHL
Processed stock: MNTR
Processed stock: VINO
Processed stock: TPTW
Processed stock: CBDY
Processed stock: TFFP
Processed stock: CLOW
Processed stock: TRCK
Processed stock: GNLN
Processed stock: TANH
Processed stock: MRPT
Processed stock: BOTH
Processed stock: PLSH
Processed stock: LGMK
Processed stock: GRLF
Processed stock: ELAB
Processed stock: ASII
Processed stock: MINM
Processed stock: FRCB
Processed stock: NVSGF
Processed stock: GMER
Processed stock: CBDW
Processed stock: PXMD
Processed stock: OILCF
Processe

...

Processed stock: NAMI
Processed stock: GOOG
Processed stock: TSMWF
Processed stock: NONOF
Processed stock: BRK-A
Processed stock: JPM-PC
Processed stock: JPM-PD
Processed stock: BML-PH
Processed stock: BAC-PE
Processed stock: BML-PJ
Processed stock: BML-PG
Processed stock: SAPGF
Processed stock: ASMLF
Processed stock: BML-PL
Processed stock: TOYOF
Processed stock: BAC-PB
Processed stock: BAC-PK
Processed stock: NVSEF
Processed stock: BABAF
Processed stock: WFC-PY
Processed stock: AZNCF
Processed stock: HBCYF
Processed stock: RYDAF
Processed stock: CILJF
Processed stock: WFC-PL
Processed stock: PCCYF
Processed stock: TTFNF
Processed stock: BHPLF
Processed stock: BUDFF
Processed stock: MBFJF
Processed stock: UNLYF
Processed stock: SNYNF
Processed stock: BPAQF
Processed stock: EADSF
Processed stock: RTPPF
Processed stock: WFC-PC
Processed stock: SNEJF
Processed stock: ABLZF
Processed stock: ABBNY
Processed stock: NTTYY
Processed stock: AIQUF
Processed stock: STOHF
Processed stock: SMFNF
P

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/THCPW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=THCPW&crumb=bhSjaOmA8q8


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADZCF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ADZCF&crumb=bhSjaOmA8q8


ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DMYY-WT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DMYY-WT&crumb=bhSjaOmA8q8


ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=bhSjaOmA8q8


In [ ]:
%tb

SystemExit: 1

# Perform RAG

In [ ]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [ ]:
query = "What are some companies that manufacture consumer hardware?"

In [ ]:
def process_query(query):
    raw_query_embedding = get_huggingface_embeddings(query)
    top_matches = pinecone_index.query(
        vector=raw_query_embedding.tolist(),
        top_k=10,
        include_metadata=True,
        namespace=namespace)
    contexts = [item['metadata']['text'] for item in top_matches['matches']]
    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query
    return augmented_query

In [ ]:
raw_query_embedding = get_huggingface_embeddings(query)

In [ ]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [ ]:
top_matches

{'matches': [{'id': 'd998e798-0b48-4ade-8815-acd0833ac50b',
              'metadata': {'Business Summary': 'Kimball Electronics, Inc. '
                                               'engages in the provision of '
                                               'electronics manufacturing, '
                                               'engineering, and supply chain '
                                               'support services to customers '
                                               'in the automotive, medical, '
                                               'and industrial end markets. '
                                               'The Company also offers '
                                               'contract manufacturing '
                                               'services, including '
                                               'engineering and supply chain '
                                               'support for the production of '
                 

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

<CONTEXT>
Kimball Electronics, Inc. engages in the provision of electronics manufacturing, engineering, and supply chain support services to customers in the automotive, medical, and industrial end markets. The Company also offers contract manufacturing services, including engineering and supply chain support for the production of electronic assemblies and other products, including non-electronic components, medical devices, medical disposables, and precision molded plastics, as well as automation, test, and inspection equipment primarily used in automotive, medical, and industrial applications. In addition, it is also involved in the production and testing of printed circuit board assemblies, as well as clean room assembly, cold chain, and product sterilization management activities. Further, the company offers design services and support, supply chain services and support, rapid prototyping and new product introduction support, product design and process validation and qualification,

# Setting up Groq for RAG

1. Get your Groq API Key [here](https://console.groq.com/keys). Through Groq, you get free access to various LLMs

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)

In [ ]:
!pip install groq
from groq import Groq
client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
client = OpenAI(
  base_url="https://api.groq.com/openai/v1",
  api_key=userdata.get("GROQ_API_KEY")
)

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
def get_response(augmented_query):
  system_prompt = f"""
  You are an expert at providing answers about stocks. Please answer my question provided.
    """

  chat_completion = client.chat.completions.create(
      model="llama-3.1-70b-versatile",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": augmented_query}
      ]
  )
  response = chat_completion.choices[0].message.content
  return response

In [ ]:
## OPENAI CHAT COMPLETION VERSION
 # system_prompt = f"""You are an expert at providing answers about stocks. Please answer my question provided.
 # """

 # llm_response = client.chat.completions.create(
 #     model="llama-3.1-70b-versatile",
 #     messages=[
 #         {"role": "system", "content": system_prompt},
 #         {"role": "user", "content": augmented_query}
 #     ]
 # )

 # response = llm_response.choices[0].message.content

In [ ]:
print(response)

Based on the provided context, the following companies manufacture or distribute consumer hardware:

1. 3M Company: Offers various consumer products including home cleaning products, retail abrasives, paint accessories, car care DIY products, picture hanging, and consumer air quality solutions.
2. Deswell Industries, Inc.: Produces electronic entertainment products, power tools, accessories, outdoor equipment, and audio equipment.
3. Forward Industries, Inc.: Designs, manufactures, sources, markets, and distributes carrying and protective solutions, including cases for portable electronic and non-electronic products.
4. Best Buy Co., Inc.: Offers a wide variety of consumer electronics and appliances, including computing and mobile phone products, home theaters, and appliances.

Additionally, companies like Kimball Electronics, Inc., Standex International Corporation, Arrow Electronics, Inc., Avnet, Inc., and Barnes Group Inc. are involved in the manufacturing or distribution of electro

In [ ]:
# install streamlit
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Build Streamlit App

%%writefile app.py

import streamlit as st
from groq import Groq
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))
index_name = "stocks"

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)


def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)

def process_query(query):
    raw_query_embedding = get_huggingface_embeddings(query)
    top_matches = pinecone_index.query(
        vector=raw_query_embedding.tolist(),
        top_k=10,
        include_metadata=True,
        namespace=namespace)
    contexts = [item['metadata']['text'] for item in top_matches['matches']]
    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query
    return augmented_query

def get_response(augmented_query):
  system_prompt = f"""
  You are an expert at providing answers about stocks. Please answer my question provided.
    """

  chat_completion = client.chat.completions.create(
      model="llama-3.1-70b-versatile",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": augmented_query}
      ]
  )
  response = chat_completion.choices[0].message.content
  return response

st.title("Stock Information App")

query = st.text_input("Enter your query:")

if st.button("Submit"):
  st.write(f"You entered: {query}")
  augmented_query = process_query(query)
  st.write(f"Augmented Query: {augmented_query}")
  response = get_response(augmented_query)
  st.write(f"Response: {response}")


Overwriting app.py


In [ ]:
pip install streamlit pyngrok python-dotenv

In [ ]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata
import os

In [ ]:
ngrok_token = userdata.get("NGROK_AUTH_TOKEN")

ngrok.set_auth_token(ngrok_token)

In [ ]:
# Define helper fcn to run Streamlit App
def run_streamlit():
  os.system("streamlit run /content/app.py --server.port 8501")
  # run app.py file and the server on port 8501

In [ ]:
# Create a new thread to run the run_streamlit() in the background
thread = Thread(target=run_streamlit)
thread.start()

In [ ]:
# Create public url for app
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)
# addr = port used
# proto = protocol, in this case http
# bind_tls = telling ngrok to provide an https external url for users

print("Public URL: ", public_url)

Public URL:  NgrokTunnel: "https://93ab-34-134-90-148.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Start Streamlit server on a specific port
!nohup streamlit run --no-reload streamlit_app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://452b-34-134-90-148.ngrok-free.app


In [ ]:
#install and run localtunnel
!npm install -g localtunnel
!streamlit run app.py & npx localtunnel --port 8501 & wget -q -O - https://loca.lt/mytunnelpassword

⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇34.134.90.148⠙

⠹⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.134.90.148:8501

⠇your url is: https://silent-crabs-lead.loca.lt
2024-12-07 22:28:01.479978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 22:28:01.506548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 22:28:01.515031: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 22:28: